In [4]:
import pandas as pd
from pandas.tseries.offsets import BDay
import numpy as np
import matplotlib.pyplot as plt
import psycopg2 as pg

from config import DATABASE_URI

plt.rcParams["figure.figsize"] = (15,10)

# Data

In [17]:
with pg.connect(DATABASE_URI) as conn:
    with conn.cursor() as cur:
        cur.execute("SELECT date, close FROM prices WHERE frequency='MINUTE' AND ticker='^GSPC'")
        result = cur.fetchall()
    
data = pd.DataFrame.from_records(result, columns=["date", "close"], coerce_float=True, index="date").squeeze()

# Daily realized volatility estimator from intraday data
results = {}
for idx, day in data.groupby(data.index.date):
    results[idx] = np.sum(np.square(np.log(day) - np.log(day.shift(1))))
    
vol = pd.Series(results)
vol = vol.reindex(pd.to_datetime(vol.index))
wk_vol = np.sqrt(vol.rolling(5).sum().dropna())
# Transform to annual percentage terms
wk_vol = wk_vol * np.sqrt(252/5) * 100

In [120]:
with pg.connect(DATABASE_URI) as conn:
    with conn.cursor() as cur:
        cur.execute("SELECT ticker, date, close FROM prices WHERE frequency='DAILY' AND ticker IN ('^GSPC', '^VIX')")
        result = cur.fetchall()

data = pd.DataFrame.from_records(result, columns=["ticker", "date", "close"], coerce_float=True, index="date")
data = data.sort_index()

spx = data[data["ticker"] == "^GSPC"]["close"]
vix = data[data["ticker"] == "^VIX"]["close"]

spx_wk_returns = (np.log(spx) - np.log(spx.shift(5))).dropna()

In [275]:
# Input features construction
x = pd.DataFrame(index=wk_vol.index)
# Realized volatility from last 4 weekly periods
x["RV-1"] = wk_vol
x["RV-2"] = wk_vol.shift(5)
x["RV-3"] = wk_vol.shift(10)
x["RV-4"] = wk_vol.shift(15)
# Last 4 end-of-week VIX values
x["VIX-1"] = vix
x["VIX-2"] = vix.shift(5)
x["VIX-3"] = vix.shift(10)
x["VIX-4"] = vix.shift(15)
# SPX returns from previous 4 weeks
x["SPX-1"] = spx_wk_returns
x["SPX-2"] = spx_wk_returns.shift(5)
x["SPX-3"] = spx_wk_returns.shift(10)
x["SPX-4"] = spx_wk_returns.shift(15)
# Standard deviation of RV over past month
x["RV-VOL"] = wk_vol.rolling(21).std()

# Calculates RV percentile rank.
# Percent of RV's in the last year that are below the last
# recorded RV value
rv_percentile = pd.Series(index=x.index, dtype=float)

for date in rv_percentile.index:
    # Get past year worth of RV data, if there isn't enough, skip
    wk_vol_interval = wk_vol.loc[date - BDay(252):date]
    if len(wk_vol_interval) < 240:
        continue
    rv_percent = stats.percentileofscore(wk_vol_interval, wk_vol.loc[date])
    rv_percentile.loc[date] = rv_percent
# 1-year RV rank
x["RV-PERCENT"] = rv_percentile

In [302]:
x = x.dropna()
# Forecasting objective: 1 week forward RV
y = wk_vol.shift(-5).dropna()

common_index = x.index.intersection(y.index)

split = int(0.80 * len(common_index))
x_train = x.loc[common_index[:split]]
y_train = y.loc[common_index[:split]]
x_test = x.loc[common_index[split:]]
y_test = y.loc[common_index[split:]]

# Model

In [ ]:
model_spec = """

"""